In [ ]:
import os
import scanpy
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import json
import warnings

from gprofiler import GProfiler

___
### Create gene name mapping (from "gene symbols" to TAIR IDs)

In [ ]:
def build_gene_mapping():

    file = "data/WT1/features.tsv.gz"
    assert os.path.exists(file), f"{file} does not exist"
    
    genes_df = pd.read_csv(file, sep='\t', header=None) # .iloc[:, 0,1]]
    genename_mapping = { row[0]: row[1] for i, row in genes_df.iterrows() }
    genename_mapping.update({ row[1]: row[0] for i, row in genes_df.iterrows() })

    return genename_mapping

In [ ]:
gene_mapping = build_gene_mapping()

In [ ]:
# Not using this anymore:
# Write gene names to a text file. 
# This will be loaded by an R script which will query Biomart for the corresponding TAIR IDs (gene IDs that start with `AT`).
# this is important for filtering the chloroplastic and mitochondrial genes which are recognized by their names.

# with open("gene_names.txt", "wt") as genes_f:
#    genes_f.write("\n".join(all_data.var_names.to_list()))

# genename_mapping = pd.read_csv("gene_symbols_to_tair_ids.txt")
# genename_mapping = { row.external_gene_name: row.tair_locus for index, row in genename_mapping.iterrows() }

gp = GProfiler(return_dataframe=True)
genename_mapping_df = gp.convert(all_data.var_names.to_list(), organism="athaliana", target_namespace="TAIR_LOCUS")
genename_mapping_df.head()

# Creamos un diccionario para usar luego:
genename_mapping = {row.incoming: row.converted for i, row in genename_mapping_df.iterrows()}

___

In [ ]:
def filter_chloroplastic_and_mitochondrial(all_data, threshold_pct=5):

    tair_ids = pd.Series(all_data.var_names).apply(lambda x: genename_mapping.get(x, x))
    
    def get_chloroplast_genes(adata: scanpy.AnnData):
        return adata.var_names[tair_ids.str.startswith("ATCG")].to_list()
    
    def get_mitochondrial_genes(adata: scanpy.AnnData):
        return adata.var_names[tair_ids.str.startswith("ATMG")].to_list()

    # Identify chloroplastic and mitocondrial genes
    chloroplast_genes     = get_chloroplast_genes(all_data)
    mitochondrial_genes   = get_mitochondrial_genes(all_data)

    # print(f"Chloroplastic genes: {chloroplast_genes}")
    # print(f"Mitochondrial genes: {mitochondrial_genes}")
    
    all_data.obs['percent_pt'] = 100 * ( all_data[:, chloroplast_genes].X.sum(axis=1)   / all_data.X.sum(axis=1) ) 
    all_data.obs['percent_mt'] = 100 * ( all_data[:, mitochondrial_genes].X.sum(axis=1) / all_data.X.sum(axis=1) )

    ## Examine cells with a percentage of protoplastic genes greater than >5%
    # all_data.obs['percent_pt'][all_data.obs['percent_pt'] > 5]
    
    # now we get rid of them
    all_data = all_data[all_data.obs['percent_pt'] < threshold_pct, :]
    
    return all_data

___

In [ ]:
def pipeline(files=["data/WT1", "data/WT2", "data/WT3"]):

    replicas = list()
    for i, file in enumerate(files):
        replicas.append(scanpy.read_10x_mtx(file, make_unique=True))
        replicas[i].obs["replica"] = f"replica_{i+1}"

    all_data = scanpy.concat(replicas); del replicas
    all_data.obs_names_make_unique()

    MIN_GENES, MIN_CELLS = 3, 200
    scanpy.pp.filter_cells(all_data, min_genes=MIN_GENES); print(all_data.shape)
    scanpy.pp.filter_genes(all_data, min_cells=MIN_CELLS); print(all_data.shape)

    all_data = filter_chloroplastic_and_mitochondrial(all_data)

    FLAVOR = "seurat_v3"
    scanpy.pp.highly_variable_genes(all_data, flavor=FLAVOR, n_top_genes=2000, min_mean=0.0125, max_mean=3, min_disp=0.5, span=1)
    all_data.var.sort_values("variances_norm", ascending=False)

    


___
### Load single cell RNA-Seq data

In [ ]:
rep1 = scanpy.read_10x_mtx("data/WT1", make_unique=True); rep1.obs["replica"] = "replica_1"; print(f"{rep1.shape=}")
rep2 = scanpy.read_10x_mtx("data/WT2", make_unique=True); rep2.obs["replica"] = "replica_2"; print(f"{rep2.shape=}")
rep3 = scanpy.read_10x_mtx("data/WT3", make_unique=True); rep3.obs["replica"] = "replica_3"; print(f"{rep3.shape=}")

all_data = scanpy.concat([rep1, rep2, rep3]); del rep1, rep2, rep3

# Ver cómo funciona esto
all_data.obs_names_make_unique()

___
Filter cells with at least 3 genes detected, filter genes that are detected in at least 200 cells.

In [ ]:
MIN_GENES, MIN_CELLS = 3, 200

scanpy.pp.filter_cells(all_data, min_genes=MIN_GENES); print(all_data.shape)
scanpy.pp.filter_genes(all_data, min_cells=MIN_CELLS); print(all_data.shape)

## Normalization

In [ ]:
# scanpy.pp.regress_out(all_data, keys='percent_pt')

___

Check the docs: https://scanpy.readthedocs.io/en/stable/generated/scanpy.pp.highly_variable_genes.html

In [ ]:
# !pip install --user scikit-misc

In [ ]:
FLAVOR = "seurat_v3"
scanpy.pp.highly_variable_genes(all_data, flavor=FLAVOR, n_top_genes=2000, min_mean=0.0125, max_mean=3, min_disp=0.5, span=1)
all_data.var.sort_values("variances_norm", ascending=False)

Write 2000 highly variable genes to file:

In [ ]:
scanpy.experimental.pp.normalize_pearson_residuals_pca(all_data, n_comps=50)

In [ ]:
pca_loadings_df = pd.DataFrame(all_data.varm['PCs'], index=all_data.var_names, columns=[f"PC{i}" for i in range(1,51)])
pca_loadings_df.to_csv("pca_loadings.csv")
pca_loadings_df.PC1[pca_loadings_df.PC1 != 0].abs().sort_values(ascending=False).head(20)

In [ ]:
scanpy.pp.normalize_total(all_data, target_sum=1e4)
scanpy.pp.log1p(all_data)
scanpy.pp.scale(all_data)

___
Top highly variable genes:

In [ ]:
# How is `variances_norm` calculated?
all_data.var.sort_values('variances_norm', ascending=False).head(20)

Genes that are not highly variable:

In [ ]:
all_data.var.sort_values('variances_norm').head(20)

In [ ]:
RANDOM_STATE = 242

In [ ]:
# plt.plot(np.cumsum(all_data.uns['pca']['variance_ratio']))
plt.plot(all_data.uns['pca']['variance']);

In [ ]:
# scanpy.pp.pca(all_data, n_comps=50, mask_var="highly_variable", random_state=RANDOM_STATE)

Let's look at what genes show up on each PC:

In [ ]:
scanpy.pl.pca_loadings(all_data, components=range(1,13))

In [ ]:
all_data.uns['pca']

In [ ]:
scanpy.pp.neighbors(all_data, n_neighbors=10, n_pcs=50, random_state=RANDOM_STATE)

In [ ]:
# !pip install louvain

In [ ]:
scanpy.tl.louvain(all_data, resolution=0.70)

## UMAP


In [ ]:
scanpy.tl.umap(all_data, random_state=RANDOM_STATE)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(16, 6))

scanpy.pl.umap(all_data, color='louvain', palette='tab20', title='UMAP Colored by Clusters', ax=ax[0], show=False)
ax[0].legend(loc='upper right', bbox_to_anchor=(0.98, 0.98), frameon=False, fontsize=6, title="Clusters",title_fontsize=10)

primary_colors = ['blue', 'red', 'yellow']
scanpy.pl.umap(all_data, color='replica', palette=primary_colors, title='UMAP Colored by replica', ax=ax[1], show=False)
ax[1].legend(loc='upper right', bbox_to_anchor=(0.98, 0.98), frameon=True, fontsize=10, title="Replicas", title_fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
data = dict()

for v in sorted(all_data.obs['louvain'].unique().astype(int)):
    data[v] = all_data[all_data.obs['louvain']==str(v)].shape[0] # /all_data.shape[0]*100

df = pd.DataFrame.from_dict(data, orient='index', columns=['cell counts'])    
df['cluster'] = df.index
df = df.reset_index(drop=True)
sns.barplot(data=df, x='cluster', y='cell counts')
plt.show()

In [ ]:
# warnings.filterwarnings("ignore")
warnings.filterwarnings("always")
scanpy.tl.rank_genes_groups(all_data, 'louvain', method='wilcoxon') # "t-test"

In [ ]:
scanpy.pl.rank_genes_groups(all_data, n_genes=20, sharey=False)

### Associate cell types to clusters

This is the list of gene markers that are hardcoded into the Seurat pipeline (they were manually generated from papers):

In [ ]:
gene_markers_by_cell_type = json.load(open("gene_markers_by_cell_type.json"))
gene_markers_by_cell_type.keys()

But I found another one online: https://biobigdata.nju.edu.cn/scplantdb/marker?species=arabidopsis_thaliana (click on button "table showing all markers to this species", top right corner, to download)

This one is more comprehensive as it includes more cell types for the leaf, so I will use it instead in what follows:

In [ ]:
gene_markers = pd.read_csv("arabidopsis_thaliana.marker_fd.csv.gz")
gene_markers = gene_markers.groupby("clusterName").apply(lambda x: x.nlargest(50, 'avg_log2FC')).reset_index(drop=True)

display(gene_markers.sample(20))
print(gene_markers.clusterName.unique())

In [ ]:
tissue_of_interest = 'Leaf'
leaf_cell_types = gene_markers.query("tissue == @tissue_of_interest").clusterName.unique()
leaf_cell_types

Here is an explanation of the different leaf cell types:

| **Cell Type**             | **Description**                                                                                 |
|---------------------------|-------------------------------------------------------------------------------------------------|
| **Mesophyll**             | Tissue in the leaf interior, responsible for photosynthesis. Contains chloroplasts.            |
| **Leaf Pavement Cell**    | Epidermal cells forming the leaf's outer layer, providing protection and structure.             |
| **Companion Cell**        | Cells in the phloem aiding nutrient transport by loading/unloading sugars into sieve tubes.     |
| **Xylem**                 | Vascular tissue transporting water and minerals from roots to leaves, composed of dead cells.   |
| **Leaf Guard Cell**       | Cells surrounding stomata, regulating gas exchange and water loss by opening/closing pores.     |
| **Phloem Parenchyma**     | Parenchyma cells in the phloem, storing and transporting nutrients laterally.                   |
| **S Phase**               | Cell cycle phase where DNA replication occurs, preparing for division.                         |
| **Vascular Tissue**       | Includes xylem and phloem, responsible for transporting water, nutrients, and food.            |
| **Sieve Element**         | Phloem cells conducting nutrients, connected by sieve plates to form sieve tubes.              |
| **Hydathodes**            | Specialized pores at leaf edges or tips involved in guttation (water exudation).               |
| **Phloem**                | Vascular tissue transporting sugars and organic nutrients throughout the plant.                |
| **G2/M Phase**            | Cell cycle phase where the cell transitions from growth (G2) to mitosis (M phase).             |
| **Spongy Mesophyll**      | Loose tissue with air spaces for gas exchange, located in the lower part of the leaf.          |
| **Palisade Mesophyll**    | Dense tissue beneath the upper epidermis, optimized for photosynthesis with many chloroplasts.  |
| **Stress Response**       | Cellular reaction to environmental stress (drought, pathogens, etc.), involving signaling or changes. |
| **Bundle Sheath**         | Cells surrounding vascular bundles, aiding in carbon fixation in C4 plants (las plantas C4 son aquellas que utilizan la vía C4 o ruta C4, un proceso de fotosíntesis que se caracteriza por tener una serie de reacciones bioquímicas para fijar el CO2 de la atmósfera).                    |
| **Leaf Epidermis**        | Outer protective layer of a leaf, includes pavement cells, guard cells, and trichomes.          |
| **Meristematic Cell**     | Undifferentiated cells in meristems capable of dividing and differentiating into other cell types. |


In [ ]:
gene_markers_by_cell_type = dict()
for cell_type in leaf_cell_types:    
    gene_markers_by_cell_type[cell_type] = gene_markers.query("clusterName == @cell_type").gene.to_list()

In [ ]:
ranked_genes_by_cluster = list()
for cluster in range(0, 15):
    ranked_genes_df = pd.DataFrame({
        'cluster': cluster,
        'gene': all_data.uns['rank_genes_groups']['names'][str(cluster)],
        'score': all_data.uns['rank_genes_groups']['scores'][str(cluster)],
        'pvals': all_data.uns['rank_genes_groups']['pvals'][str(cluster)],
    })
    ranked_genes_by_cluster.append(ranked_genes_df)
    
ranked_genes_by_cluster = pd.concat(ranked_genes_by_cluster)

In [ ]:
N_GENES = 50
topN_deg_genes_by_cluster = ranked_genes_by_cluster.groupby("cluster").apply(lambda x: x.nlargest(N_GENES, 'score')).reset_index(drop=True)
topN_deg_genes_by_cluster['tair_id'] = topN_deg_genes_by_cluster.gene.apply(lambda x: genename_mapping.get(x, x))

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

@interact
def show_top_genes_for_cluster(cluster=widgets.IntSlider(min=1,max=15)):
  return topN_deg_genes_by_cluster.query("cluster == @cluster")

In [ ]:
len(gene_markers_by_cell_type['Bundle sheath'])

In [ ]:
gene_markers_by_cluster = { cluster: topN_deg_genes_by_cluster.query("cluster == @cluster").gene.apply(lambda x: genename_mapping.get(x, x)).to_list() for cluster in range(15) }

In [ ]:
{ cell: len(genes) for cell, genes in gene_markers_by_cell_type.items() }

In [ ]:
contingency_table = {}

for group1, elements1 in gene_markers_by_cell_type.items():
    row = {}
    for group2, elements2 in gene_markers_by_cell_type.items():
        intersection = set(elements1) & set(elements2)
        row[group2] = len(intersection)
    contingency_table[group1] = row

df = pd.DataFrame(contingency_table)

plt.figure(figsize=(8, 6))  # Set the figure size
sns.heatmap(df, annot=True, cmap="Blues", fmt="d", cbar_kws={'label': 'Intersection Count'})

plt.title('Cell type to cell type contingency table')
plt.tight_layout()
plt.show()

In [ ]:
contingency_table = {}

for group1, elements1 in gene_markers_by_cluster.items():
    row = {}
    for group2, elements2 in gene_markers_by_cell_type.items():
        intersection = set(elements1) & set(elements2)
        row[group2] = len(intersection)
    contingency_table[group1] = row

df = pd.DataFrame(contingency_table)

plt.figure(figsize=(8, 6))
sns.heatmap(df, annot=True, cmap="Blues", fmt="d", cbar_kws={'label': 'Intersection size (200 is perfect match)'})

plt.title('Cell type to cell type contingency table')
plt.tight_layout()
plt.show()

In [ ]:
mesophyll_clusters = ['5', '6', '7', '8']

subset_data = all_data[all_data.obs['louvain'].isin(mesophyll_clusters)].copy()

scanpy.pp.pca(subset_data, n_comps=20)
scanpy.pp.neighbors(subset_data)
scanpy.tl.umap(subset_data)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6, 6))

scanpy.pl.umap(subset_data, color='louvain', 
           palette='Set2',
           title='UMAP Colored by Clusters within the Mesophylls', ax=ax, show=False)

In [ ]:
pca_loadings_df = pd.DataFrame(subset_data.varm['PCs'], index=subset_data.var_names, columns=[f"PC{i}" for i in range(1,21)])

In [ ]:
def query_for_go_terms(genes):
    terms = enriched = gp.profile(
        organism="athaliana",
        query=list(genes),
        sources=["GO:BP", "GO:MF", "GO:CC", "KEGG"],
        no_iea=False,         # Include electronic annotations
        user_threshold=0.05,
    )
    return terms

In [ ]:
go_terms = list()

for pc in range(1, 5):
    top_genes = pca_loadings_df[f"PC{pc}"].abs().sort_values(ascending=False).head(500)
    go_terms_for_pc = query_for_go_terms(top_genes.index).assign(PC=pc).drop(['significant', 'query'], axis=1)
    go_terms.append(go_terms_for_pc)

go_terms = pd.concat(go_terms).reset_index().sort_values("p_value")

In [ ]:
idx = go_terms.groupby('native')['p_value'].idxmin()

In [ ]:
go_terms.loc[idx].sort_values("p_value").head(20)

___

# Compare with Seurat

In [ ]:
cell_names = all_data.obs_names

In [ ]:
cluster_assignments = all_data.obs["louvain"]

# import pandas as pd
cell_cluster_df = pd.DataFrame({
    "cell": cell_names,
    "cluster_scanpy": cluster_assignments
})

In [ ]:
cell_cluster = cell_cluster_df["cluster_scanpy"].astype(int)

In [ ]:
seurat_cluster_142 = pd.read_csv("/mnt/rodrigo/Postdoc/scrna-pipeline/output/output_seed142/cells_by_cluster.csv").iloc[:,1:]

In [ ]:
seurat_cluster_142 = pd.read_csv("/mnt/rodrigo/Postdoc/scrna-pipeline/output/output_seed142/cells_by_cluster.csv").iloc[:,1:]
seurat_cluster_142.columns = ['cell', 'cluster_seurat']
seurat_cluster_242 = pd.read_csv("/mnt/rodrigo/Postdoc/scrna-pipeline/output/output_seed242/cells_by_cluster.csv").iloc[:,1:]
seurat_cluster_242.columns = ['cell', 'cluster_seurat']

seurat_cluster_142 = seurat_cluster_142.set_index("cell").iloc[:,0]
seurat_cluster_242 = seurat_cluster_242.set_index("cell").iloc[:,0]

In [ ]:
contingency_matrix = pd.crosstab(cell_cluster, seurat_cluster_142)

In [ ]:
contingency_matrix.apply(lambda x: x / contingency_matrix.apply(sum,axis=1)).round(2)*100

In [ ]:
fraction_matrix = (contingency_matrix / contingency_matrix.apply(sum,axis=0)).round(4) * 100

In [ ]:
styled_df = fraction_matrix.round(2).style.map(lambda x: "font-weight: bold" if x > 50 else "").format("{:.2f}")
styled_df

In [ ]:
seurat_cluster_142.sample(60)

Our clusters:

In [ ]:
genes_in_my_clusters = pd.DataFrame(all_data.uns['rank_genes_groups']['names']).head(200)

In [ ]:
genes_in_my_clusters

In [ ]:
for i in range(genes_in_my_clusters.shape[0]):
    for j in range(genes_in_my_clusters.shape[1]):
        genes_in_my_clusters.iloc[i,j] = genename_mapping.get(genes_in_my_clusters.iloc[i,j], genes_in_my_clusters.iloc[i,j])

In [ ]:
genes_in_my_clusters = { i: set(genes_in_my_clusters[str(i)]) for i in range(genes_in_my_clusters.shape[1])}

In [ ]:
genes_in_my_clusters

Clusters in paper:

In [ ]:
sheet_names = [  "C0C4C10C11 healthy mesophyl", 
  "C1 responsive epidermal cells",
  "C2 vascular S cells",
  "C5",
  "C8 responsive mesophyl cells",
  "C9 healthy epidermal cells",
  "C13",
  "C16 guard cells"
]

genes_in_cluster = dict()

for cluster in sheet_names:
    supp_material_clusters = pd.read_excel("~/Postdoc/Papers/AT_PST_ScienceDirect_files_06Nov2024_13-40-25.581/1-s2.0-S2590346223002043-mmc5.xlsx", sheet_name=cluster)
    genes = supp_material_clusters[~supp_material_clusters['Gene model'].isna()]["Gene model"]
    genes_in_cluster[cluster] = set(genes.to_list())

In [ ]:
conting_matrix = np.zeros((len(genes_in_my_clusters), len(genes_in_cluster)), int)

In [ ]:
for i, (cluster_i, genes_i) in enumerate(genes_in_my_clusters.items()):
    for j, (cluster_j, genes_j) in enumerate(genes_in_cluster.items()):
        conting_matrix[i,j] = len(genes_i.intersection(genes_j))

In [ ]:
pd.DataFrame(conting_matrix).T

## Gene ontology term enrichment

In [ ]:
gp = GProfiler(return_dataframe=True)
results = {}
for cluster_name, genes in genes_in_my_clusters.items():
    query_for_go_terms(genes)
    results[cluster_name] = enriched

In [ ]:
all_go_results = pd.concat([results[i].assign(cluster=i) for i in range(len(results))])

In [ ]:
all_go_results[all_go_results.name.apply(lambda x: 'response' in x)].sort_values("p_value")

In [ ]:
cluster, gene_term = 11, 'GO:0006952'
cluster, gene_term = 6, "GO:0043207"
cluster, gene_term = 4, "GO:0009607"

term_details = gp.convert(
    query=gene_term,
    organism="athaliana",
    target_namespace="ENSG",
)

In [ ]:
"AT2G45180" in term_details[term_details.converted.isin(genes_in_my_clusters[cluster])].converted

# Compare to Seurat

In [ ]:
open("2000_highly_variable_genes.txt", "w").write("\n".join(all_data.var['highly_variable'][all_data.var['highly_variable']].index.to_list()))
highly_variable_scanpy = set(pd.read_csv("2000_highly_variable_genes.txt").iloc[:,0])

highly_variable_seurat = set(pd.read_csv("2000_highly_variable_genes_seurat.txt").iloc[:,0])

len(highly_variable_scanpy & highly_variable_seurat) # &: intersection

In [ ]:
highly_variable_seurat_seed142 = set(pd.read_csv("2000_highly_variable_genes_seurat_seed142.txt").iloc[:,0])
highly_variable_seurat_seed242 = set(pd.read_csv("2000_highly_variable_genes_seurat_seed242.txt").iloc[:,0])

In [ ]:
len(highly_variable_seurat_seed142 & highly_variable_seurat_seed242)